<a href="https://colab.research.google.com/github/massone99/visione_artificiale_colab_notebooks/blob/main/ObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Esercitazione su object detection**
Nell'esercitazione odierna utilizzeremo l'architettura YOLO per rilevare i pezzi degli scacchi nelle immagini del database [Chess Pieces Dataset](https://public.roboflow.com/object-detection/chess-full).

Faremo uso del framework **TensorFlow**, sfruttando la libreria open-source **Keras** appositamente progettata per permettere una rapida prototipazione di reti neurali profonde.

# **Operazioni preliminari**
Eseguendo la cella sottostante tutto il materiale necessario per lo svolgimento dell'esercitazione verrà scaricato sulla macchina remota. Alla fine dell'esecuzione selezionare il tab **Files** per verificare che tutto sia stato scaricato correttamente.

In [ ]:
!wget http://bias.csr.unibo.it/VR/Esercitazioni/DBs/ObjectDetection/ChessPiecesDataset_416x416_aug.zip

!unzip -q /content/ChessPiecesDataset_416x416_aug.zip

!rm /content/ChessPiecesDataset_416x416_aug.zip

## **KerasCV**
La libreria [**KerasCV**](https://keras.io/keras_cv/) mette a disposizione componenti specifiche per svolgere task di *Computer Vision* come modelli pre-addestrati, metriche e layer.

La cella seguente permette di installare l'ultima versione di KerasCV.

In [ ]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv

# **Import delle librerie**
Per prima cosa è necessario eseguire l'import delle librerie utilizzate durante l'esecitazione.

In [ ]:
import os
import tensorflow as tf
import numpy as np
import keras_cv
import cv2
import random
import math
from tensorflow import keras
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

# **Functioni di utilità**
Tramite la cella sottostante è possibile definire alcune funzioni di utilità usate durante l'esercitazione:
- **plot_images_with_xywh_bounding_boxes** visualizza un *mini-batch* di immagini con le corrispondenti *bounding box*;
- **plot_images_with_y_preds** visulizza un *mini-batch* di immagini con le corrispondenti *prediction*;
- **parse_yolov8_annotation** legge da un file di testo le annotazioni di un'immagine nel formato YOLOv8;
- **load_dataset_info** carica in memoria i percorsi assoluti di tutte le immagini e le rispettive annotazioni di un dataset;
- **from_rel_xywh_to_xywh** converte le annotazioni dal formato relativo al formato assoluto date in input le dimensioni delle immagini;
- **load_dataset_element** carica in memoria un'immagine dato in input il percorso in cui è memorizzara e la associa alle corrispondenti *bounding box*;
- **prepare_dataset** prepara il dataset dato in input caricando le immangini e applicando alcune operazioni di preprocessing tra cui il ridimensionamento delle immagini alle dimensioni di input del modello;
- **plot_history** visualizza il grafico delle loss sui dataset di training e validation;
- **compute_mean_average_precision** calcola la *mean Average Precision* (mAP) date in input le *bounding box* vere e quelle predette dalla rete.

In [ ]:
def plot_images_with_xywh_bounding_boxes(images,boxes,class_ids,class_labels,image_per_row=4,show_labels=True,confidences=None):
  class_colors = plt.cm.hsv(np.linspace(0, 1, len(class_labels)+1)).tolist()

  image_count=len(images)
  row_count=math.ceil(image_count/image_per_row)
  col_count=image_per_row

  _, axs = plt.subplots(nrows=row_count, ncols=col_count,figsize=(18, 4*row_count),squeeze=False)
  for r in range(row_count):
      for c in range(col_count):
        axs[r,c].axis('off')

  for i in range(image_count):
    r = i // image_per_row
    c = i % image_per_row

    axs[r,c].imshow(images[i])
    for box_idx in range(len(boxes[i])):
        box=boxes[i][box_idx]
        class_idx=class_ids[i][box_idx]
        color =class_colors[class_idx]
        xmin=box[0]
        ymin=box[1]
        w=box[2]
        h=box[3]
        axs[r,c].add_patch(plt.Rectangle((xmin,ymin), w, h, color=color, fill=False, linewidth=2))
        if show_labels:
          label ='{}'.format(class_labels[class_idx])
          if confidences is not None:
            label+=' {:.2f}'.format(confidences[i][box_idx])
          axs[r,c].text(xmin, ymin, label, size='large', color='white', bbox={'facecolor':color, 'alpha':1.0})

def plot_images_with_y_preds(images,y_preds,class_labels,image_per_row=4,show_labels=True):
  image_count=images.shape[0]
  plot_images_with_xywh_bounding_boxes(images,
                                      [y_preds['boxes'][i,:y_preds['num_detections'][i]] for i in range(image_count)],
                                      [y_preds['classes'][i,:y_preds['num_detections'][i]] for i in range(image_count)],
                                      class_labels,
                                      image_per_row=image_per_row,
                                      show_labels=show_labels,
                                      confidences=[y_preds['confidence'][i,:y_preds['num_detections'][i]] for i in range(image_count)])

def parse_yolov8_annotation(txt_file):
    yolov8_boxes = []
    class_ids = []
    with open(txt_file) as file:
      for line in file:
        splitted_line = line.split()
        class_ids.append(int(splitted_line[0]))
        rcx = float(splitted_line[1])
        rcy = float(splitted_line[2])
        rw = float(splitted_line[3])
        rh = float(splitted_line[4])
        rxmin=rcx-rw/2
        rymin=rcy-rh/2
        yolov8_boxes.append([rxmin, rymin, rw, rh]) #rel_xywh

    return yolov8_boxes, class_ids

def load_dataset_info(txt_file_list,image_path):
  image_path_list = []
  image_yolov8_box_list = []
  image_class_id_list = []
  for txt_file in txt_file_list:
      yolov8_boxes, class_ids = parse_yolov8_annotation(txt_file)
      image_path_list.append(image_path+os.path.basename(os.path.splitext(txt_file)[0])+'.jpg')
      image_yolov8_box_list.append(yolov8_boxes)
      image_class_id_list.append(class_ids)

  return image_path_list,image_yolov8_box_list,image_class_id_list

def from_rel_xywh_to_xywh(dataset_rel_xywh_bboxes,image_shape):
  image_height=image_shape[0]
  image_width=image_shape[1]

  dataset_xywh_bboxes=[]
  for image_bboxes in dataset_rel_xywh_bboxes:
    image_xywh_bboxes=[]
    for image_bbox in image_bboxes:
      xmin=image_bbox[0]*image_width
      ymin=image_bbox[1]*image_height
      w=image_bbox[2]*image_width
      h=image_bbox[3]*image_height
      image_xywh_bboxes.append([xmin, ymin, w, h])
    dataset_xywh_bboxes.append(image_xywh_bboxes)

  return dataset_xywh_bboxes

def load_dataset_element(image_path, bboxes,classes):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    bounding_boxes = {
        'classes': tf.cast(classes, dtype=tf.float32),
        'boxes': bboxes,
    }
    return {'images': tf.cast(image, tf.float32), 'bounding_boxes': bounding_boxes}

def prepare_dataset(dataset,batch_size,model_input_size):
  resizer = keras.Sequential(layers=[keras_cv.layers.Resizing(model_input_size,model_input_size,pad_to_aspect_ratio=True,bounding_box_format='xywh')])

  ds = dataset.map(load_dataset_element, num_parallel_calls=tf.data.AUTOTUNE)
  ds = ds.shuffle(dataset.cardinality())
  ds = ds.ragged_batch(batch_size, drop_remainder=True)
  ds = ds.map(resizer, num_parallel_calls=tf.data.AUTOTUNE)
  ds = ds.map(lambda inputs: (inputs['images'], inputs['bounding_boxes']), num_parallel_calls=tf.data.AUTOTUNE)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

def plot_history(history):
  fig, ax1 = plt.subplots(figsize=(10, 8))

  epoch_count=len(history.history['loss'])

  line1,=ax1.plot(range(1,epoch_count+1),history.history['loss'],label='train_loss',color='orange')
  ax1.plot(range(1,epoch_count+1),history.history['val_loss'],label='val_loss',color = line1.get_color(), linestyle = '--')
  ax1.set_xlim([1,epoch_count])
  ax1.set_ylim([0, max(max(history.history['loss']),max(history.history['val_loss']))])
  ax1.set_ylabel('loss',color = line1.get_color())
  ax1.tick_params(axis='y', labelcolor=line1.get_color())
  ax1.set_xlabel('Epochs')
  _=ax1.legend(loc='lower left')

def compute_mean_average_precision(y_true, y_pred):
  coco_metrics=keras_cv.metrics.BoxCOCOMetrics(bounding_box_format='xywh',evaluate_freq=1)
  coco_metrics.update_state(y_true, y_pred)
  return coco_metrics.result(force=True)['MaP'].numpy()

#YOLOv8
YOLOv8 è l'ultimo modello basato sull'architettura YOLO (*You Only Look Once*) in grado di individuare in tempo reale oggetti multipli all'interno di un'immagine con un elevato livello di accuratezza. Quest'ultima versione è in grado di supportare diversi tipi di *task* tra cui: *detection*, segmentazione e classificazione.

## **Creazione del modello**
Il codice seguente crea un'istanza della classe [**YOLOV8Detector**](https://keras.io/api/keras_cv/models/tasks/yolo_v8_detector/) (che implementa il modello YOLOv8 per svolgere *task* di *object detection*) utilizzando il metodo [**from_preset**](https://keras.io/api/keras_cv/models/tasks/yolo_v8_detector/#frompreset-method) che permette di istanziare un modello con una configurazione predefinita e con pesi pre-addestrati.

I parametri *preset* e *load_weights* permettono di caricare i pesi ottenuti utilizzando in fase di addestramento il daset della competizione [**PascalVOC 2012**](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/) contenente immagini RGB con oggetti apartenenti a 20 classi: (*aeroplane*, *bicycle*, *bird*, *boat*, *bottle*, *bus*, *car*, *cat*, *chair*, *cow*, *diningtable*, *dog*, *horse*, *motorbike*, *person*, *pottedplant*, *sheep*, *sofa*, *train*, *tv/monitor*). Con tali pesi il modello è in grado di ottenere una *mean Average Precision* (mAP) pari a 0.45 sul dataset di test della competizione.

Il parametro *bounding_box_format* permette di definire il formato di input/output delle *bounding box*. Con la stringa **'xywh'** le *bounding box* saranno rappresentate tramite:
- le coordinate in pixel dell'angolo superiore sinistro $(x,y)$;
- la larghezza ($w$) e l'altezza  ($h$) in pixel della box.


In [ ]:
yolov8_detector = keras_cv.models.YOLOV8Detector.from_preset(preset='yolo_v8_m_pascalvoc',load_weights=True,bounding_box_format='xywh')

## **Visualizzazione del modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura del modello appena caricato.

In [ ]:
yolov8_detector.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
tf.keras.utils.plot_model(yolov8_detector,show_shapes=True, show_layer_names=True)

# **Esercizio 1**
Utilizzare il modello appena istanziato per individuare gli oggetti presenti su una o più immagini prese dal web.

A tal fine:

1. scaricare una o più immagini dal web contenenti una o più istanze delle classi del problema (*aeroplane*, *bicycle*, *bird*, *boat*, *bottle*, *bus*, *car*, *cat*, *chair*, *cow*, *diningtable*, *dog*, *horse*, *motorbike*, *person*, *pottedplant*, *sheep*, *sofa*, *train*, *tvmonitor*);
2. trasferire le immagini su **Colab** utilizzando la funzione *Upload* del tab **Files**;
3. caricare un'immagine nella variabile *my_image* (per farlo utilizzare la funzione [**imread(...)**](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html) della libreria [**Matplotlib**](https://matplotlib.org/));
4. se necessario, riscalare l'immagine così che la dimensione maggiore sia pari alla dimensione di input della rete ($512\times512$). Per farlo utilizzare la funzione **resize** di OpenCV;
5. se necessario, aggiungere un bordo nero alla dimensione minore così da portarla alla dimensione di input della rete ($512\times512$) senza distorcere l'immagine. Per farlo utilizzare la funzione [**pad**](https://numpy.org/doc/stable/reference/generated/numpy.pad.html) di NumPy;
6. creare un *mini-batch* contenente una singola immagine da dare in input alla rete. Per farlo utilizzare la funzione [**expand_dims**](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html) di Numpy per aggiungere una dimensione unitaria all'immagine;
7. effettuare la *detection* degli oggetti richiamando il metodo [**predict**](https://keras.io/api/models/model_training_apis/#predict-method) del modello e memorizzando l'output nella variabile *my_y_preds*.

In [ ]:
# Define class labels and model input size
class_labels = ['aeroplane', 'bicycle', 'bird', 'boat',
                'bottle', 'bus', 'car', 'cat',
                'chair', 'cow', 'diningtable', 'dog',
                'horse', 'motorbike', 'person', 'pottedplant',
                'sheep', 'sofa', 'train', 'tvmonitor']
model_input_size = 512

# Define image file path
image_file_path = "test_images_for_yolo/"

# Load the original image
original_image = mpimg.imread(image_file_path + "horse_sofa_person.jpg")

# Calcola il rapporto di ridimensionamento
max_dimension = 512
current_height, current_width = original_image.shape[:2]
if current_height > current_width:
    scale_factor = max_dimension / current_height
else:
    scale_factor = max_dimension / current_width

# Resize the image
resized_image = cv2.resize(original_image, None, fx=scale_factor, fy=scale_factor)

# Calculate padding to reach the desired size (512x512)
vertical_padding = max(0, max_dimension - resized_image.shape[0])
horizontal_padding = max(0, max_dimension - resized_image.shape[1])
top_padding = vertical_padding // 2
bottom_padding = vertical_padding - top_padding
left_padding = horizontal_padding // 2
right_padding = horizontal_padding - left_padding

# Add black padding
padded_image = np.pad(resized_image, ((top_padding, bottom_padding), (left_padding, right_padding), (0, 0)), mode='constant', constant_values=0)

# Create a mini-batch containing a single image
image_batch = np.expand_dims(padded_image, axis=0)

# Perform object detection
# Assuming you have already created the yolov8_detector object as mentioned in your previous code
my_y_preds = yolov8_detector.predict(image_batch)

## **Output del modello**
Eseguendo la cella seguente si può vedere che il modello restituisce un dizionario (**dict**) contenente le chiavi: *boxes*, *confidence*, *classes* e *num_detections*. Ognuna di queste chiavi ha associato un array di NumPy:
- *boxes* contiene, per ogni immagine del *mini-batch* di input, un massimo di 100 *bounding box* ognuna rappresentata da 4 valori $(x,y,w,h)$;
- *confidence* contiene, per ogni immagine del *mini-batch* di input, un massimo di 100 valori che rappresentano il livello di confidenza con cui la rete ha individuato l'oggetto contenuto nella *bounding box* corrispondente;
- *classes* contiene, per ogni immagine del *mini-batch* di input, un massimo di 100 valori che rappresentano l'ID della classe associata all'oggetto  contenuto nella *bounding box* corrispondente;
- *num_detections* contiene, per ogni immagine del *mini-batch* di input, il numero di oggetti effettivamente individuati ($\le100$).

In [ ]:
print('Tipo dell\'output del modello: ',type(my_y_preds))
print('Chiavi contenute nel dizionario: ',my_y_preds.keys())
print('Shape di boxes: ',my_y_preds['boxes'].shape)
print('Shape di confidence: ',my_y_preds['confidence'].shape)
print('Shape di classes: ',my_y_preds['classes'].shape)
print('Shape di num_detections: ',my_y_preds['num_detections'].shape)

## **Visualizzazione del risultato**
Richiamando la funzione di utilità **plot_images_with_y_preds** è possibile visualizzare gli oggetti trovati.

In [ ]:
plot_images_with_y_preds(image_batch,my_y_preds,class_labels,image_per_row=1,show_labels=True)

# **Dataset**
In questa esercitazione useremo il [Chess Pieces Dataset](https://public.roboflow.com/object-detection/chess-full) per addestrare il modello YOLOv8 ad individuare i pezzi del gioco degli scacchi.

Il dataset consiste in un insieme di immagini RGB. Ogni immagine ha associato un file di annotazioni in formato testuale (TXT) contenente
la classe e le bounding box (memorizzate nel formato YOLOv8) associate a tutti i pezzi degli scacchi presenti nell'immagine (*ground truth*).

Eseguendo la cella sottostante, i percorsi assoluti di tutti i file con le annotazioni per i set di training, validation e test vengono caricati in memoria.

In [ ]:
# Classi del problema
chess_class_labels = ['none',
                      'black-bishop', 'black-king', 'black-knight', 'black-pawn', 'black-queen', 'black-rook',
                      'white-bishop', 'white-king', 'white-knight', 'white-pawn', 'white-queen', 'white-rook']

original_image_size=(416,416)

# Percorsi in cui si trovano i set di training, validation e test
train_base_path='/content/416x416_aug/train'
val_base_path='/content/416x416_aug/valid'
test_base_path='/content/416x416_aug/test'

# Percorsi in cui si trovano le immagini dei set di training, validation e test
train_path_images = train_base_path+'/images/'
val_path_images = val_base_path+'/images/'
test_path_images = test_base_path+'/images/'

# Percorsi in cui si trovano le annotazioni dei set di training, validation e test
train_path_annotations = train_base_path+'/labels/'
val_path_annotations = val_base_path+'/labels/'
test_path_annotations = test_base_path+'/labels/'

# Lista dei file delle annotazioni di training
train_txt_files = sorted(
    [
        os.path.join(train_path_annotations, file_name)
        for file_name in os.listdir(train_path_annotations)
        if file_name.endswith(".txt")
    ]
)

# Lista dei file delle annotazioni di validation
val_txt_files = sorted(
    [
        os.path.join(val_path_annotations, file_name)
        for file_name in os.listdir(val_path_annotations)
        if file_name.endswith(".txt")
    ]
)

# Lista dei file delle annotazioni di test
test_txt_files = sorted(
    [
        os.path.join(test_path_annotations, file_name)
        for file_name in os.listdir(test_path_annotations)
        if file_name.endswith(".txt")
    ]
)

print('Numero dei file di training: ',len(train_txt_files))
print('Numero dei file di validation: ',len(val_txt_files))
print('Numero dei file di test: ',len(test_txt_files))

La cella sottostante permette di caricare in memoria i percorsi assoluti delle immagini e le rispettive annotazioni per i set di training, validation e test.

In [ ]:
train_image_path_list,train_yolov8_box_list,train_image_class_id_list=load_dataset_info(train_txt_files,train_path_images)
val_image_path_list,val_yolov8_box_list,val_image_class_id_list=load_dataset_info(val_txt_files,val_path_images)
test_image_path_list,test_yolov8_box_list,test_image_class_id_list=load_dataset_info(test_txt_files,test_path_images)

Il formato YOLOv8 con cui sono memorizzate le *bounding box* dei pezzi degli scacchi riporta $x$, $y$, $w$ e $h$ come valori relativi (nell'intervallo $[0;1]$) rispetto alla dimensione dell'immagine a cui sono associate.

La cella sottostante stampa a video i valori $(x,y,w,h)$ in formato relativo della *bounding box* associata alla prima immagine del training set.

In [ ]:
print(train_yolov8_box_list[0])

Ai fini dell'addestramento del modello e della visualizzazione dei risultati è conveniente convertire le *bounding box*  da valori relativi a valori assoluti nel formato **'xywh'**. Per fare ciò è sufficiente utilizzare la funzione di utilità **from_rel_xywh_to_xywh** passandogli in input, oltre alle *bounding box* nel formato YOLOv8, le dimensioni delle immagini del dataset ($416\times416$).

In [ ]:
train_xywh_box_list=from_rel_xywh_to_xywh(train_yolov8_box_list,original_image_size)
val_xywh_box_list=from_rel_xywh_to_xywh(val_yolov8_box_list,original_image_size)
test_xywh_box_list=from_rel_xywh_to_xywh(test_yolov8_box_list,original_image_size)

La cella sottostante stampa a video i valori $(x,y,w,h)$ in formato assoluto della *bounding box* associata alla prima immagine del training set.

In [ ]:
print(train_xywh_box_list[0])

## **Visualizzazione dei dati**
Per capire meglio la natura e la difficoltà del problema che affronteremo può essere molto utile visualizzare alcune immagini di esempio. Eseguendo la cella sottostante verranno visualizzate quattro immagini del training set scelte casualmente (con le rispettive annotazioni).

In [ ]:
image_count=4
random_idx=random.randint(0,len(train_image_path_list)-image_count)

plot_images_with_xywh_bounding_boxes([plt.imread(train_image_path_list[i]) for i in range(random_idx,random_idx+image_count)],
                                     train_xywh_box_list[random_idx:random_idx+image_count],
                                     train_image_class_id_list[random_idx:random_idx+image_count],
                                     chess_class_labels,
                                     image_per_row=4,
                                     show_labels=False)

## **Preparazione dei dati**
Quando si usano database di grosse dimensioni non sempre è possibile e conveniente caricarli interamente in memoria. Per ovviare a questo problema è possibile utilizzare la classe [**Dataset**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) di TensorFlow che permette di gestire in maniera efficiente i dati caricandoli in memoria solo all'occorrenza.

Il metodo [**from_tensor_slices**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) crea un'istanza della classe **Dataset** partendo da un insieme di tensori che posseggono la prima dimensione con la medesima cardinalità.

Avendo memorizzato le informazioni dei set di training, validation e test in delle liste, prima di tutto dobbiamo convertirle in tensori. Le funzioni [**tf.constant**](https://www.tensorflow.org/api_docs/python/tf/constant) e  [**tf.ragged.constant**](https://www.tensorflow.org/api_docs/python/tf/ragged/constant) di TensorFlow creano rispettivamente un tensore e un tensore irregolare (ovvero che può avere delle dimensioni a lunghezza variabile) a partire da una lista. I tensori irregolari sono necessari in quanto il numero di *bounding box* può variare per ogni immagine.

In [ ]:
chess_train_data = tf.data.Dataset.from_tensor_slices((tf.constant(train_image_path_list),
                                                      tf.ragged.constant(train_xywh_box_list),
                                                      tf.ragged.constant(train_image_class_id_list)))

chess_val_data = tf.data.Dataset.from_tensor_slices((tf.constant(val_image_path_list),
                                                     tf.ragged.constant(val_xywh_box_list),
                                                     tf.ragged.constant(val_image_class_id_list)))

chess_test_data = tf.data.Dataset.from_tensor_slices((tf.constant(test_image_path_list),
                                                     tf.ragged.constant(test_xywh_box_list),
                                                     tf.ragged.constant(test_image_class_id_list)))

La funzione **prepare_dataset** permette di preparare i dataset prima di utilizzarli nelle fasi di addestramento e valutazione delle prestazioni.

In [ ]:
batch_size = 4

chess_train_ds=prepare_dataset(chess_train_data,batch_size,model_input_size)
chess_val_ds=prepare_dataset(chess_val_data,batch_size,model_input_size)
chess_test_ds=prepare_dataset(chess_test_data,chess_test_data.cardinality().numpy(),model_input_size)

La cella sottostante riporta il numero di *mini-batch* creati per ognuno dei tre dataset.

In [ ]:
print('Mini-batch di training: ',chess_train_ds.cardinality().numpy())
print('Mini-batch di validation: ',chess_val_ds.cardinality().numpy())
print('Mini-batch di test: ',chess_test_ds.cardinality().numpy())

Iterando su un Dataset è possibile ottenere un *mini-batch* per volta.

In [ ]:
images, bounding_boxes =next(iter(chess_train_ds))

print('Shape del mini-batch delle immagini: ',images.shape)
print('Shape del mini-batch delle bounding box: ',bounding_boxes['boxes'].shape)
print('Shape del mini-batch delle classi associate alle bounding box: ',bounding_boxes['classes'].shape)

Grazie alla funzione **plot_images_with_xywh_bounding_boxes** è possibile visualizzare a video le immagini appartenenti a un *mini-batch* con evidenziate le *bounding box* associate.

In [ ]:
plot_images_with_xywh_bounding_boxes(images/255,
                                     bounding_boxes['boxes'].to_list(),
                                     tf.dtypes.cast(bounding_boxes['classes'],tf.int32).to_list(),
                                     chess_class_labels,
                                     image_per_row=4,
                                     show_labels=False)

# **Transfer learning**
L'addestramento di YOLOv8 su un nuovo problema, richiede un training set etichettato di notevoli dimensioni.

In alternativa al training da zero, possiamo utilizzare un modello pre-addestrato su un altro problema di *object detection* e effettuare [*fine-tuning*](https://en.wikipedia.org/wiki/Fine-tuning_(deep_learning)) sul nostro problema.


## **Caricamento del modello pre-addestrato**
Il metodo [**from_preset**](https://keras.io/api/keras_cv/models/backbones/yolo_v8/#frompreset-method) della classe [**YOLOV8Backbone**](https://keras.io/api/keras_cv/models/backbones/yolo_v8/#yolov8backbone-class) permette di istanziare una YOLOv8 caricando i pesi pre-addestrati su uno specifico dataset per poi utilizzarli in un altro contesto o su un altro *task*.

Eseguendo la cella sottostante verranno caricati i pesi ottenuti utilizzando in fase di addestramento il daset [**COCO**](https://cocodataset.org/).

In [ ]:
yolov8_backbone = keras_cv.models.YOLOV8Backbone.from_preset('yolo_v8_s_backbone_coco',load_weights=True)

## **Creazione del modello**
Il codice seguente crea un'istanza della classe **YOLOV8Detector** utilizzando il *backbone* con i pesi pre-addestrati caricati nella cella precedente e impostando il numero di classi (*num_classes*) in base al problema che vogliamo affrontare.

In [ ]:
yolov8_chess_detector = keras_cv.models.YOLOV8Detector(backbone=yolov8_backbone,num_classes=len(chess_class_labels),bounding_box_format='xywh')

## **Visualizzazione del modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura del modello appena caricato.

In [ ]:
yolov8_chess_detector.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
tf.keras.utils.plot_model(yolov8_chess_detector,show_shapes=True, show_layer_names=True)

## **Configurare il modello per il training**
Con il metodo **compile** è possibile configurare il modello per la fase di training dati in input:
- un ottimizzatore (*optimizer*);
- la *loss* di classificazione (*classification_loss*);
- la *loss* di regressione delle *bounding box* (*box_loss*).

In particolare come *loss* di regressione delle *bounding box* viene utilizzata la **'ciou'** (*Complete Intersection over Union*), una variante della *Intersection over Union* (IoU).

In [ ]:
# Configura il modello per il training
yolov8_chess_detector.compile(optimizer='adam', classification_loss='binary_crossentropy', box_loss='ciou')

# **Training**
Ora siamo pronti per effettuare il *fine-tuning* della YOLOv8 utilizzando le immagini e le annotazioni dei dataset di training (*chess_train_ds*) e validation (*chess_val_ds*) per un numero di epoche pari a *epoch_count*.

In [ ]:
# Numero di epoche di addestramento
epoch_count = 30

Per evitare che il modello faccia *overfitting* sul dataset di training, si può utilizzare la classe di *callback* [**EarlyStopping**](https://keras.io/api/callbacks/early_stopping/#earlystopping-class) che permette di interrompere l'addestramento quando la metrica monitorata smette di migliorare. I parametri da impostare sono:
- *monitor* la metrica da monitoriare;
- *patience* il numero di epoche consecutive in cui la metrica monitorata non migliora da attendere prima di interrompere l'addestramento;
- *restore_best_weights* permette di reimpostare i pesi all'epoca in la metrica monitorata ha ottenuto il valore migliore.

In [ ]:
patience=5
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

Nella cella seguente viene richiamato il metodo [**fit**](https://keras.io/api/models/model_training_apis/#fit-method) che esegue l'intera fase di addestramento in maniera automatica monitorando costantemente la *loss function* su training e validation set.

In [ ]:
history = yolov8_chess_detector.fit(chess_train_ds,validation_data=chess_val_ds,epochs=epoch_count,callbacks=[early_stop])

Il metodo **fit** restituisce un oggetto (*history*) che contiene per ogni epoca i valori della *loss function* sia per il training sia per il validation set.

Richiamando la funzione **plot_history** è possibile graficare tali valori.

In [ ]:
plot_history(history)

# **Valutazione delle prestazioni sul test set**
Per verificare l'effettiva capacità di generalizzazione del modello appena addestrato, è fondamentale misurarne le prestazioni sul dataset di test.

## **Test set**
Iterando sul dataset di test (*chess_test_ds*) è possibile estrarre immagini (*test_x*) e annotazioni (*test_y_true*).

In [ ]:
test_x, test_y_true = next(iter(chess_test_ds.take(1)))
print('Shape del mini-batch di test: ',test_x.shape)

## **Object detection**
Eseguendo la cella sottostante, si effettuerà la *detection* sulle immagini del test set (*test_x*) richiamando il metodo **predict** del modello appena addestrato.

In [ ]:
test_y_pred = yolov8_chess_detector.predict(test_x)

## **Visualizzazione dei risultati**
Richiamando la funzione di utilità **plot_images_with_y_preds** è possibile visualizzare i pezzi trovati sulle immagini del test set.

In [ ]:
plot_images_with_y_preds(test_x/255,test_y_pred,chess_class_labels,image_per_row=5,show_labels=False)

Visivamente si possono notare gli ottimi risultati ottenuti.

Per avere una valutazione delle prestazioni più obiettiva è opportuno calcolare la *mean Average Precision* (mAP).

Per farlo è sufficiente richiamare la funzione **compute_mean_average_precision** passandogli in input il *ground truth* del test set e le *prediction* (*test_y_pred*) restituite dal modello.

In [ ]:
test_map=compute_mean_average_precision(test_y_true, test_y_pred)
print(test_map)

L'mAP ottenuto risulta molto elevato confermando le ottime prestazioni della YOLOv8 nell'individuare correttamente i pezzi degli scacchi.

# **Esercizio 2**
Risolvere un altro problema di *object detection* scelto su [**Roboflow**](https://public.roboflow.com/object-detection) modificando il codice visto durante l'esercitazione.